In [1]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [3]:
from langchain.document_loaders import WebBaseLoader

# ссылка на страницу из Википедии про Ганнибала
url = "https://ru.wikipedia.org/wiki/%D0%93%D0%B0%D0%BD%D0%BD%D0%B8%D0%B1%D0%B0%D0%BB,_%D0%90%D0%B1%D1%80%D0%B0%D0%BC_%D0%9F%D0%B5%D1%82%D1%80%D0%BE%D0%B2%D0%B8%D1%87"

# Определите лоадер
loader = WebBaseLoader(url)
data = loader.load()

In [27]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from utils import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'                                          или cuda - если через видеокарту
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)
db_embed = FAISS.from_documents(texts, hf_embeddings_model)

# retriever = db_embed.as_retriever()

In [25]:
                             #  1 вариант retriever(извлечение)
#  Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db_embed.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=5,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
                              #  2 вариант retriever(извлечение)
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Попробуем BM25 - алгоритм поиска по ключевым словам. Может искать лучше, но не поймёт запрос на другом языке.
# Также не сможет искать, если запрос на обывательском языке, а документы на профессиональном
bm25 = BM25Retriever.from_documents(texts[:70])  # Эмбеддинги ему не нужны
bm25.k = 5  # Так можно задать количество возвращаемых документов


                           #  3 вариант retriever(извлечение) Объединяем bm25 и retriever (первый и второй вариант)
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# Если один ретривер плохо справляется, то можно использовать целый ансамбль и даже каждому вес назначить
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],  # список ретриверов
    weights=[
        0.3,
        0.7,
    ],  # веса, на которые домножается скор документа от каждого ретривера
)

In [26]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_self_ask_with_search_agent, Tool
from langchain_community.utilities import SerpAPIWrapper

tool = create_retriever_tool(
    ensemble_retriever, # наш ретривер
    "search_web", # имя инструмента
    "Searches and returns data from page", # описание инструмента подается в ЛЛМ
)



tools = [tool]

In [35]:
from langchain import hub

prompt = hub.pull("hwchase17/react")   # hwchase17/react или hwchase17/openai-tools-agent
# """Please ensure that your response does not exceed 70 characters. And respond in Russian language.""" -добавляю чтобы ответ был меньше 70 символов
prompt += """Please ensure that your response does not exceed 70 characters. And respond in Russian language."""
print(prompt)

C:\Users\fRomanSt\AppData\Local\Programs\Python\Python39\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}Please ensure that your response does not exceed 70 characters. And respond in Russian language.'


In [36]:
from langchain.agents import create_react_agent
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor


agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

In [37]:
import pandas as pd
from tqdm import tqdm

# Загружаем датасет с вопросами про Ганнибала
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1107866/gannibal.csv")


In [38]:
answers = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['question']):
    answer = agent_executor.invoke({"input": text_input})     ###  Получим ответ от агента
    answers.append(answer['output']) # Добавляем ответ в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 10/10 [00:22<00:00,  2.22s/it]


In [39]:
print(answers)


['Евдокия Андреевна Диопер', 'Да.', '85', '1727', 'Он был ранен в голову во время Войны четверного альянса.', '100 рублей в год.', 'Со Швецией.', 'Генерал-аншеф.', 'Христина-Регина фон Шёберг.', '7']


In [40]:
df['answer'] = answers # Создаём новый столбец из ответов модели

In [42]:
df.to_csv('step6_solution.csv', index=False) # Сохраняем файл
df

,question,answer
0,На ком в начале 1731 года Абрам Ганнибал женил...,Евдокия Андреевна Диопер
1,Является ли А.С. Пушкин сыном Абрама Ганнибала...,Да.
2,Сколько лет прожил Абрам Ганнибал? Ответ дайте...,85
3,В каком году Абрам Ганнибал был отправлен в Си...,1727
4,Куда был ранен Абрам Ганнибал участвуя в Войне...,Он был ранен в голову во время Войны четверног...
5,Какой размер жалования Абрам Ганнибал получал ...,100 рублей в год.
6,Разграничением земель с какой страной в 1745 г...,Со Швецией.
7,В каком звании Абрам Ганнибал был отправлен в ...,Генерал-аншеф.
8,Как звали вторую жену Абрама Ганнибала?,Христина-Регина фон Шёберг.
9,Сколько всего детей Абрама Ганнибала дожили до...,7
